In [1]:
import anndata as ad 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import scanpy as sc

from lab_scripts.utils import utils
utils.change_directory_to_repo()

In [2]:
new_atac = ad.read_h5ad('data/official/common/openproblems_bmmc_multiome_phase1v2/openproblems_bmmc_multiome_phase1v2.manual_formatting.output_rna.h5ad')

In [3]:
new_atac.obs['batch'].value_counts()

s3d7     6094
s1d2     6069
s1d1     5616
s2d4     5456
s2d5     4395
s3d10    3909
s1d3     3875
s2d1     3811
s3d6     1771
s3d3     1496
Name: batch, dtype: int64

In [3]:
old_atac = ad.read_h5ad('data/official/common_v1/openproblems_bmmc_multiome_phase1/openproblems_bmmc_multiome_phase1.manual_formatting.output_rna.h5ad')

In [4]:
selected_batches = ['s1d1', 's1d2', 's2d1', 's2d4']

In [5]:
new_batches = new_atac.obs["batch"].astype("string")
selected_idx = new_batches.apply(lambda batch: batch in selected_batches).values
new_atac = new_atac[selected_idx]

In [6]:
old_batches = old_atac.obs["batch"].astype("string")
selected_idx = old_batches.apply(lambda batch: batch in selected_batches).values
old_atac = old_atac[selected_idx]

In [7]:
new_atac.obs['batch'].value_counts()

s1d2    6069
s1d1    5616
s2d4    5456
s2d1    3811
Name: batch, dtype: int64

In [8]:
old_atac.obs['batch'].value_counts()

s1d2    6069
s1d1    5616
s2d4    5456
s2d1    3811
Name: batch, dtype: int64

In [9]:
new_names = set(new_atac.obs.index)

In [10]:
old_names = set(old_atac.obs.index)

In [11]:
new_names == old_names

True

In [13]:
new_atac.obs

,pct_counts_mt,n_counts,n_genes,size_factors,phase,cell_type,pseudotime_order_GEX,batch,pseudotime_order_ATAC,is_train
TAGTTGTCACCCTCAC-1-s1d1,1.061008,1508.0,1022.0,0.453484,S,Naive CD20+ B,NaN,s1d1,NaN,True
CTATGGCCATAACGGG-1-s1d1,0.604230,1655.0,1081.0,0.455631,G2M,CD14+ Mono,NaN,s1d1,NaN,True
CCGCACACAGGTTAAA-1-s1d1,0.650069,7230.0,3304.0,2.435348,G2M,CD8+ T,NaN,s1d1,NaN,True
TCATTTGGTAATGGAA-1-s1d1,0.812274,1108.0,793.0,0.347226,G2M,CD8+ T,NaN,s1d1,NaN,True
ACCACATAGGTGTCCA-1-s1d1,1.674770,1851.0,1219.0,0.534205,G2M,CD16+ Mono,NaN,s1d1,NaN,True
...,...,...,...,...,...,...,...,...,...,...
CTCCTGAGTTTATTCG-s2d4,0.132275,2268.0,1476.0,0.658597,G2M,CD14+ Mono,NaN,s2d4,NaN,True
TAGCCTGAGTGAGAGC-s2d4,0.064893,1541.0,1096.0,0.534516,S,G/M prog,NaN,s2d4,NaN,True
GGGCTAACAGCCAGTT-s2d4,0.148258,1349.0,890.0,0.346754,G2M,CD14+ Mono,NaN,s2d4,NaN,True
TTACCTGTCTACTAAG-s2d4,0.047170,4240.0,1919.0,1.143241,S,Erythroblast,0.926222,s2d4,0.729032,True


In [14]:
old_atac.obs

,n_genes_by_counts,pct_counts_mt,n_counts,n_genes,size_factors,phase,cell_type,pseudotime_order_GEX,batch,pseudotime_order_ATAC,is_train
TAGTTGTCACCCTCAC-1-s1d1,1022,1.061008,1508.0,1022.0,0.453484,S,Naive CD20+ B,NaN,s1d1,NaN,True
CTATGGCCATAACGGG-1-s1d1,1081,0.604230,1655.0,1081.0,0.455631,G2M,CD14+ Mono,NaN,s1d1,NaN,True
CCGCACACAGGTTAAA-1-s1d1,3304,0.650069,7230.0,3304.0,2.435348,G2M,CD8+ T,NaN,s1d1,NaN,True
TCATTTGGTAATGGAA-1-s1d1,793,0.812274,1108.0,793.0,0.347226,G2M,CD8+ T,NaN,s1d1,NaN,True
ACCACATAGGTGTCCA-1-s1d1,1219,1.674770,1851.0,1219.0,0.534205,G2M,CD16+ Mono,NaN,s1d1,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...
AAACCGAAGGACAATG-s2d4,778,0.378788,1320.0,778.0,0.195915,G2M,Normoblast,0.939465,s2d4,0.776010,True
TAGTAAGCACAAAGCG-s2d4,571,0.107181,933.0,571.0,0.190195,G2M,Erythroblast,0.932991,s2d4,0.833617,True
AGTCTTGCATAAACCT-s2d4,1045,0.570613,1402.0,1045.0,0.375832,G2M,CD14+ Mono,NaN,s2d4,NaN,True
CTCTAAGCAAAGCCTC-s2d4,1090,0.088692,2255.0,1090.0,0.511494,G2M,Erythroblast,0.930954,s2d4,0.749031,True


In [15]:
names = list(new_atac.obs.index)

In [16]:
new_idx = np.argsort(names)

In [17]:
new_atac = new_atac[new_idx]

In [18]:
old_names = list(old_atac.obs.index)

In [19]:
old_idx = np.argsort(old_names)

In [20]:
old_atac = old_atac[old_idx]

In [24]:
new_atac.layers['counts'][-1].toarray()

array([[0., 0., 0., ..., 2., 0., 0.]])

In [25]:
old_atac.layers['counts'][-1].toarray()

array([[0., 0., 0., ..., 2., 0., 0.]])

In [36]:
(new_atac.layers['counts'] - old_atac.layers['counts']).sum()

0.0

In [29]:
new_gex = ad.read_h5ad('data/official/common/openproblems_bmmc_multiome_phase1v2/openproblems_bmmc_multiome_phase1v2.manual_formatting.output_rna.h5ad')

In [30]:
old_gex = ad.read_h5ad('data/official/common_v1/openproblems_bmmc_multiome_phase1/openproblems_bmmc_multiome_phase1.manual_formatting.output_rna.h5ad')

In [31]:
new_batches = new_gex.obs["batch"].astype("string")
selected_idx = new_batches.apply(lambda batch: batch in selected_batches).values
new_gex = new_gex[selected_idx]

In [32]:
old_batches = old_gex.obs["batch"].astype("string")
selected_idx = old_batches.apply(lambda batch: batch in selected_batches).values
old_gex = old_gex[selected_idx]

In [33]:
new_gex

View of AnnData object with n_obs × n_vars = 20952 × 13431
    obs: 'pct_counts_mt', 'n_counts', 'n_genes', 'size_factors', 'phase', 'cell_type', 'pseudotime_order_GEX', 'batch', 'pseudotime_order_ATAC', 'is_train'
    var: 'gene_ids', 'feature_types', 'genome'
    uns: 'dataset_id', 'organism'
    obsm: 'X_pca', 'X_umap'
    layers: 'counts'

In [34]:
old_gex

View of AnnData object with n_obs × n_vars = 20952 × 13431
    obs: 'n_genes_by_counts', 'pct_counts_mt', 'n_counts', 'n_genes', 'size_factors', 'phase', 'cell_type', 'pseudotime_order_GEX', 'batch', 'pseudotime_order_ATAC', 'is_train'
    var: 'gene_ids', 'feature_types', 'genome'
    uns: 'dataset_id', 'organism'
    obsm: 'X_pca', 'X_umap'
    layers: 'counts'

In [38]:
new_names = list(new_gex.obs.index)
new_idx = np.argsort(new_names)
new_gex = new_gex[new_idx]

In [39]:
old_names = list(old_gex.obs.index)
old_idx = np.argsort(old_names)
old_gex = old_gex[old_idx]

In [40]:
new_gex.obs

,pct_counts_mt,n_counts,n_genes,size_factors,phase,cell_type,pseudotime_order_GEX,batch,pseudotime_order_ATAC,is_train
AAACAGCCAACACTTG-2-s1d2,1.410743,1843.0,1221.0,0.719166,G2M,NK,NaN,s1d2,NaN,False
AAACAGCCAATAGTCT-s2d4,0.322789,1549.0,1024.0,0.422408,G2M,Naive CD20+ B,NaN,s2d4,NaN,True
AAACAGCCAATTAAGG-1-s1d1,1.057872,1607.0,1088.0,0.475175,G2M,CD4+ T naive,NaN,s1d1,NaN,True
AAACAGCCACCAGGTT-2-s1d2,2.068618,1982.0,1305.0,0.730747,G2M,NK,NaN,s1d2,NaN,False
AAACAGCCAGCTACGT-s2d4,0.181324,1103.0,799.0,0.315154,G2M,CD8+ T,NaN,s2d4,NaN,True
...,...,...,...,...,...,...,...,...,...,...
TTTGTTGGTGTAATAC-s2d4,0.119261,1677.0,1086.0,0.447510,G2M,Erythroblast,0.916678,s2d4,0.697933,True
TTTGTTGGTTAGACCA-2-s1d2,1.383311,2241.0,1200.0,0.700421,G2M,CD8+ T,NaN,s1d2,NaN,False
TTTGTTGGTTCGGTAA-1-s1d1,1.844355,2223.0,1347.0,0.671612,G2M,NK,NaN,s1d1,NaN,True
TTTGTTGGTTTGGGTA-4-s2d1,0.250209,1199.0,828.0,0.324995,G2M,CD4+ T naive,NaN,s2d1,NaN,True


In [41]:
old_gex.obs

,n_genes_by_counts,pct_counts_mt,n_counts,n_genes,size_factors,phase,cell_type,pseudotime_order_GEX,batch,pseudotime_order_ATAC,is_train
AAACAGCCAACACTTG-2-s1d2,1221,1.410743,1843.0,1221.0,0.719166,G2M,NK,NaN,s1d2,NaN,False
AAACAGCCAATAGTCT-s2d4,1024,0.322789,1549.0,1024.0,0.422408,G2M,Naive CD20+ B,NaN,s2d4,NaN,True
AAACAGCCAATTAAGG-1-s1d1,1088,1.057872,1607.0,1088.0,0.475175,G2M,CD4+ T naive,NaN,s1d1,NaN,True
AAACAGCCACCAGGTT-2-s1d2,1305,2.068618,1982.0,1305.0,0.730747,G2M,NK,NaN,s1d2,NaN,False
AAACAGCCAGCTACGT-s2d4,799,0.181324,1103.0,799.0,0.315154,G2M,CD8+ T,NaN,s2d4,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...
TTTGTTGGTGTAATAC-s2d4,1086,0.119261,1677.0,1086.0,0.447510,G2M,Erythroblast,0.916678,s2d4,0.697933,True
TTTGTTGGTTAGACCA-2-s1d2,1200,1.383311,2241.0,1200.0,0.700421,G2M,CD8+ T,NaN,s1d2,NaN,False
TTTGTTGGTTCGGTAA-1-s1d1,1347,1.844355,2223.0,1347.0,0.671612,G2M,NK,NaN,s1d1,NaN,True
TTTGTTGGTTTGGGTA-4-s2d1,828,0.250209,1199.0,828.0,0.324995,G2M,CD4+ T naive,NaN,s2d1,NaN,True


In [42]:
(new_gex.X - old_gex.X).sum()

0.0

In [43]:
(new_gex.layers['counts'] - old_gex.layers['counts']).sum()

0.0

In [51]:
(new_gex.obs.index != new_atac.obs.index).sum()

0